# Accion Microsft

In [232]:
import requests
import pandas as pd
import plotly.express as px

API_KEY = "f8t2eKKxjdTt7LiuD0Na5nMB0aYdpdgi"
TICKER = "MSFT"

# Nuevo endpoint “stable” para precios EOD completos (open, high, low, close, volumen, etc.)
url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={TICKER}&apikey={API_KEY}"

headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    print("ERROR:", resp.status_code)
    print("Respuesta del servidor:", resp.text)
    raise RuntimeError(f"Error al conectar con FMP (stable endpoint): {resp.status_code}")

data = resp.json()

# aquí puede que la estructura JSON sea distinta: por ejemplo “historical” dentro de un campo “historical” o directamente raíz
if "historical" in data:
    hist = data["historical"]
else:
    # si la respuesta es directamente un arreglo
    hist = data

df = pd.DataFrame(hist)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

fig = px.line(df, x="date", y="close",
              title=f"Precio de cierre diario — {TICKER} (endpoint stable)",
              labels={"date": "Fecha", "close": "Precio (USD)"},
              template="plotly_white")
fig.show()

display(df.head(), df.tail())


,symbol,date,open,high,low,close,volume,change,changePercent,vwap
1255,MSFT,2020-10-19,220.42,222.30,213.72,214.22,27625841,-6.19,-2.81000,217.6650
1254,MSFT,2020-10-20,215.80,217.37,213.09,214.65,22753511,-1.15,-0.53290,215.2275
1253,MSFT,2020-10-21,213.12,216.92,213.12,214.80,22724906,1.68,0.78829,214.4900
1252,MSFT,2020-10-22,213.93,216.06,211.70,214.89,22351500,0.96,0.44874,214.1450
1251,MSFT,2020-10-23,215.03,216.28,213.16,216.23,18879608,1.20,0.55806,215.1750


,symbol,date,open,high,low,close,volume,change,changePercent,vwap
4,MSFT,2025-10-13,516.41,516.41,511.68,514.05,14284238,-2.36,-0.45700,514.6375
3,MSFT,2025-10-14,510.23,515.28,506.00,513.57,14684300,3.35,0.65461,511.2700
2,MSFT,2025-10-15,514.96,517.19,510.00,513.43,14694700,-1.53,-0.29711,513.8950
1,MSFT,2025-10-16,512.58,516.85,508.13,511.61,15559600,-0.97,-0.18924,512.2925
0,MSFT,2025-10-17,509.04,515.48,507.31,513.58,19205931,4.54,0.89187,511.3525


In [233]:
df = df[df['date'] >= '2025-06-12']

fig = px.line(df, x="date", y="close",
              title=f"Precio de cierre diario — {TICKER} (endpoint stable)",
              labels={"date": "Fecha", "close": "Precio (USD)"},
              template="plotly_white")
fig.show()

In [234]:
df = df[['close','date']]
df['price'] = df['close']
df.drop(columns=['close'], inplace=True)
df.set_index('date', inplace=True)
df

,price
date,
2025-06-12,478.87
2025-06-13,474.96
2025-06-16,479.14
2025-06-17,478.04
2025-06-18,480.24
...,...
2025-10-13,514.05
2025-10-14,513.57
2025-10-15,513.43


# Scaler

In [235]:
from sklearn.preprocessing import MinMaxScaler

In [236]:
scaler = MinMaxScaler()
price_scaled = scaler.fit_transform(df[['price']])
df_scaled = pd.DataFrame({'Date': df.index, 'price_scaled': price_scaled.flatten()})
df_scaled.set_index('Date', inplace=True)
df_scaled

,price_scaled
Date,
2025-06-12,0.064436
2025-06-13,0.000000
2025-06-16,0.068886
2025-06-17,0.050758
2025-06-18,0.087014
...,...
2025-10-13,0.644199
2025-10-14,0.636289
2025-10-15,0.633982


# Window

In [237]:
import numpy as np
import pandas as pd
from typing import Tuple, Optional

def make_sliding_windows_pd(
    data: pd.Series | pd.DataFrame,
    col: Optional[str] = None,
    lags: int = 30,
    horizon: int = 1,
    step: int = 1,
    return_index: bool = True
) -> tuple:
    """
    Ventanas deslizantes a partir de una Serie/DataFrame con índice temporal.

    Parámetros
    ----------
    data : pd.Series | pd.DataFrame
        Serie univariada o DataFrame con una columna objetivo.
    col : str | None
        Nombre de la columna si 'data' es DataFrame. Ignorado si es Series.
    lags : int
        Tamaño de ventana (número de rezagos).
    horizon : int
        Pasos futuros a predecir (1 = next-step; >1 = multi-step directo).
    step : int
        Avance entre ventanas consecutivas.
    return_index : bool
        Si True, devuelve índices de fin de X y de y (útil para alinear fechas).

    Returns
    -------
    X : np.ndarray, shape (n_samples, lags)
    y : np.ndarray, shape (n_samples, horizon)
    x_end_index : pd.Index (opcional)
        Índice (fechas) del último punto incluido en cada ventana X.
    y_end_index : pd.Index (opcional)
        Índice (fechas) del último target de cada y (t+lags+horizon-1).
    """
    # Selección de serie
    if isinstance(data, pd.DataFrame):
        if col is None:
            if data.shape[1] != 1:
                raise ValueError("Pasa 'col' si el DataFrame tiene >1 columna.")
            series = data.iloc[:, 0]
        else:
            series = data[col]
    else:
        series = data

    # Orden y limpieza mínima
    series = series.sort_index()
    series = series.dropna()

    idx = series.index
    s = series.to_numpy(dtype=float)
    n = len(s)

    max_start = n - lags - horizon
    if max_start < 0:
        empty_X = np.empty((0, lags), dtype=float)
        empty_y = np.empty((0, horizon), dtype=float)
        return (empty_X, empty_y, idx[:0], idx[:0]) if return_index else (empty_X, empty_y)

    starts = np.arange(0, max_start + 1, step, dtype=int)
    X = np.empty((len(starts), lags), dtype=float)
    y = np.empty((len(starts), horizon), dtype=float)

    # Índices de referencia
    x_end_index = idx[starts + lags - 1]
    y_end_index = idx[starts + lags + horizon - 1]

    for i, st in enumerate(starts):
        X[i] = s[st : st + lags]
        y[i] = s[st + lags : st + lags + horizon]

    return (X, y, x_end_index, y_end_index) if return_index else (X, y)


In [238]:
X, y, x_idx, y_idx = make_sliding_windows_pd(
    data=df_scaled, col="price_scaled",
    lags=7, horizon=1, step=1, return_index=True
)

# Train/Test

In [239]:
n = len(X)

if n < 30:
    raise ValueError(f"Necesitas al menos 30 muestras (tienes {n}) para 15 val + 15 test.")

n_val  = 15
n_test = 15
n_train = n - n_val - n_test
if n_train <= 0:
    raise ValueError(f"El train quedaría vacío (n_train={n_train}). Aumenta datos o reduce ventanas.")

X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test,  y_test  = X[n_train+n_val:n_train+n_val+n_test], y[n_train+n_val:n_train+n_val+n_test]

print(f"Train={n_train}, Val={n_val}, Test={n_test}")


Train=52, Val=15, Test=15


In [240]:
display(X_train, y_train)
print(f'Train shape {X_train.shape}')
display(X_val, y_val)
print(f'Val shape {X_val.shape}')
display(X_test, y_test)
print(f'Test shape {X_test.shape}')

array([[0.06443639, 0.        , 0.06888596, 0.05075808, 0.08701384,
        0.04021094, 0.18193804],
       [0.        , 0.06888596, 0.05075808, 0.08701384, 0.04021094,
        0.18193804, 0.2496704 ],
       [0.06888596, 0.05075808, 0.08701384, 0.04021094, 0.18193804,
        0.2496704 , 0.28526697],
       [0.05075808, 0.08701384, 0.04021094, 0.18193804, 0.2496704 ,
        0.28526697, 0.37063283],
       [0.08701384, 0.04021094, 0.18193804, 0.2496704 , 0.28526697,
        0.37063283, 0.34574819],
       [0.04021094, 0.18193804, 0.2496704 , 0.28526697, 0.37063283,
        0.34574819, 0.36997363],
       [0.18193804, 0.2496704 , 0.28526697, 0.37063283, 0.34574819,
        0.36997363, 0.2816414 ],
       [0.2496704 , 0.28526697, 0.37063283, 0.34574819, 0.36997363,
        0.2816414 , 0.2658207 ],
       [0.28526697, 0.37063283, 0.34574819, 0.36997363, 0.2816414 ,
        0.2658207 , 0.39353988],
       [0.37063283, 0.34574819, 0.36997363, 0.2816414 , 0.2658207 ,
        0.39353988, 0.3

array([[0.2496704 ],
       [0.28526697],
       [0.37063283],
       [0.34574819],
       [0.36997363],
       [0.2816414 ],
       [0.2658207 ],
       [0.39353988],
       [0.3750824 ],
       [0.35695452],
       [0.47050099],
       [0.4370468 ],
       [0.46736981],
       [0.46242584],
       [0.50856955],
       [0.50527357],
       [0.60547132],
       [0.5782795 ],
       [0.5784443 ],
       [0.4995056 ],
       [0.50939354],
       [0.59195781],
       [0.63859591],
       [0.61865524],
       [0.61980883],
       [0.63085036],
       [0.96473303],
       [0.80998682],
       [1.        ],
       [0.86997363],
       [0.82366513],
       [0.75609756],
       [0.77587343],
       [0.77142386],
       [0.89452868],
       [0.75181279],
       [0.78312459],
       [0.74505603],
       [0.69446276],
       [0.57366513],
       [0.50692156],
       [0.48253131],
       [0.5318062 ],
       [0.48286091],
       [0.44627554],
       [0.52373105],
       [0.57152274],
       [0.522

Train shape (52, 7)


array([[0.52373105, 0.57152274, 0.52290705, 0.49703362, 0.50082399,
        0.54400132, 0.33025709],
       [0.57152274, 0.52290705, 0.49703362, 0.50082399, 0.54400132,
        0.33025709, 0.38299275],
       [0.52290705, 0.49703362, 0.50082399, 0.54400132, 0.33025709,
        0.38299275, 0.38645353],
       [0.49703362, 0.50082399, 0.54400132, 0.33025709, 0.38299275,
        0.38645353, 0.41875412],
       [0.50082399, 0.54400132, 0.33025709, 0.38299275, 0.38645353,
        0.41875412, 0.42930125],
       [0.54400132, 0.33025709, 0.38299275, 0.38645353, 0.41875412,
        0.42930125, 0.57580751],
       [0.33025709, 0.38299275, 0.38645353, 0.41875412, 0.42930125,
        0.57580751, 0.66578774],
       [0.38299275, 0.38645353, 0.41875412, 0.42930125, 0.57580751,
        0.66578774, 0.56163481],
       [0.38645353, 0.41875412, 0.42930125, 0.57580751, 0.66578774,
        0.56163481, 0.5777851 ],
       [0.41875412, 0.42930125, 0.57580751, 0.66578774, 0.56163481,
        0.5777851 , 0.5

array([[0.38299275],
       [0.38645353],
       [0.41875412],
       [0.42930125],
       [0.57580751],
       [0.66578774],
       [0.56163481],
       [0.5777851 ],
       [0.55191167],
       [0.70814107],
       [0.65079103],
       [0.56476599],
       [0.57992749],
       [0.52851022],
       [0.60151615]])

Val shape (15, 7)


array([[0.55191167, 0.70814107, 0.65079103, 0.56476599, 0.57992749,
        0.52851022, 0.60151615],
       [0.70814107, 0.65079103, 0.56476599, 0.57992749, 0.52851022,
        0.60151615, 0.65326302],
       [0.65079103, 0.56476599, 0.57992749, 0.52851022, 0.60151615,
        0.65326302, 0.70847067],
       [0.56476599, 0.57992749, 0.52851022, 0.60151615, 0.65326302,
        0.70847067, 0.73747528],
       [0.57992749, 0.52851022, 0.60151615, 0.65326302, 0.70847067,
        0.73747528, 0.6720501 ],
       [0.52851022, 0.60151615, 0.65326302, 0.70847067, 0.73747528,
        0.6720501 , 0.69858273],
       [0.60151615, 0.65326302, 0.70847067, 0.73747528, 0.6720501 ,
        0.69858273, 0.88348715],
       [0.65326302, 0.70847067, 0.73747528, 0.6720501 , 0.69858273,
        0.88348715, 0.80784443],
       [0.70847067, 0.73747528, 0.6720501 , 0.69858273, 0.88348715,
        0.80784443, 0.82218194],
       [0.73747528, 0.6720501 , 0.69858273, 0.88348715, 0.80784443,
        0.82218194, 0.7

array([[0.65326302],
       [0.70847067],
       [0.73747528],
       [0.6720501 ],
       [0.69858273],
       [0.88348715],
       [0.80784443],
       [0.82218194],
       [0.7818062 ],
       [0.5932762 ],
       [0.64419908],
       [0.63628873],
       [0.63398154],
       [0.60398813],
       [0.63645353]])

Test shape (15, 7)


# FFNN

In [241]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import pandas as pd
from pandas.tseries.offsets import BDay
import plotly.graph_objects as go

SEED = 63
tf.random.set_seed(SEED)

In [242]:
lags = X.shape[1]
horizon = y.shape[1] if y.ndim == 2 else 1  

model = keras.Sequential()
model.add(layers.Input(shape=(lags,)))          # capa de entrada
model.add(layers.Dense(64, activation='relu')) # ocultas
model.add(layers.Dense(64,  activation='relu'))
model.add(layers.Dense(32,  activation='relu'))
model.add(layers.Dense(32,  activation='relu'))
model.add(layers.Dense(8,  activation='relu'))
model.add(layers.Dense(8,  activation='relu'))
model.add(layers.Dense(horizon, activation='linear'))  # salida: horizon

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='mse',
    metrics=['mae']
)

In [243]:
es = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    verbose=1,
    callbacks=[es],
    shuffle=False
)

Epoch 1/200


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 477ms/step - loss: 0.3550 - mae: 0.5648 - val_loss: 0.2798 - val_mae: 0.5195
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.3065 - mae: 0.5235 - val_loss: 0.2461 - val_mae: 0.4862
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 0.2724 - mae: 0.4923 - val_loss: 0.2148 - val_mae: 0.4531
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.2384 - mae: 0.4590 - val_loss: 0.1851 - val_mae: 0.4193
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.2063 - mae: 0.4254 - val_loss: 0.1562 - val_mae: 0.3835
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 0.1750 - mae: 0.3896 - val_loss: 0.1280 - val_mae: 0.3449
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.1444 - mae: 0.3511 - val_loss: 0.1009 - val_mae: 0.3032
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - loss: 0.1146 - mae: 0.3087 - val_loss: 0.0734 - val_mae: 0.2540
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.0847 - mae: 0.2

In [244]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {test_loss:.6f} | Test MAE: {test_mae:.6f}")

y_pred_test = model.predict(X_test)

Test MSE: 0.006944 | Test MAE: 0.058724
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step


In [245]:
EPOCHS_WF = 80
PATIENCE_WF = 8

y_pred_walk = []
models_used = []  

for i in range(n_test):
    print(f' ================================ Ejecucion de run {i} ================================ ')
    X_tr_i = X[:n_train + i]
    y_tr_i = y[:n_train + i]

    n_tr_i = len(X_tr_i)
    n_val_i = max(1, int(n_tr_i * 0.10))
    X_tr_core, y_tr_core = X_tr_i[:-n_val_i], y_tr_i[:-n_val_i]
    X_val_i,   y_val_i   = X_tr_i[-n_val_i:], y_tr_i[-n_val_i:]

    tf.random.set_seed(SEED + i)  
    model_i = model

    model_i.fit(
        X_tr_core, y_tr_core,
        validation_data=(X_val_i, y_val_i),
        epochs=EPOCHS_WF,
        batch_size=32,
        verbose=1,
        callbacks=[es],
        shuffle=False,
    )

    yhat = model_i.predict(X_test[i:i+1], verbose=0).ravel()[0]
    y_pred_walk.append(yhat)
    models_used.append(model_i)  # opcional

y_pred_walk = np.array(y_pred_walk).reshape(-1, 1)  

 ================================ Ejecucion de run 0 ================================ 
Epoch 1/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 225ms/step - loss: 0.0069 - mae: 0.0530 - val_loss: 0.0093 - val_mae: 0.0681
Epoch 2/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0068 - mae: 0.0529 - val_loss: 0.0097 - val_mae: 0.0702
Epoch 3/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0067 - mae: 0.0529 - val_loss: 0.0101 - val_mae: 0.0719
Epoch 4/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0067 - mae: 0.0533 - val_loss: 0.0101 - val_mae: 0.0721
Epoch 5/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0066 - mae: 0.0533 - val_loss: 0.0099 - val_mae: 0.0711
Epoch 6/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0066 - mae: 0.0531 - val_loss: 0.0096 - val_mae: 0.0699
Epoch 7/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0066 - mae: 0.0528 - val_loss: 0.0095 - val_mae: 0.0694
Epoch 8/80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0066 - mae: 0.0527 - val_loss: 0.0096 - val_mae:

In [246]:
y_idx_train = y_idx[:n_train]
y_idx_val   = y_idx[n_train:n_train+n_val]
y_idx_test  = y_idx[n_train+n_val:n_train+n_val+n_test]

In [247]:
try:
    df_comp = pd.DataFrame({
        "Date": y_idx_test,
        "Real": y_test.ravel(),
        "Estatica": y_pred_test.ravel(),   
        "WalkForward": y_pred_walk.ravel() 
    }).set_index("Date")

    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=df_comp.index, y=df_comp["Real"], name="Real"))
    fig1.add_trace(go.Scatter(x=df_comp.index, y=df_comp["Estatica"], name="Estática"))
    fig1.add_trace(go.Scatter(x=df_comp.index, y=df_comp["WalkForward"], name="Walk-Forward"))
    fig1.update_layout(
        title=f"MSFT (scaled) — Real vs Estática vs Walk-Forward (test)",
        xaxis_title="Fecha", yaxis_title="price_scaled", hovermode="x unified",
        template="plotly_white"
    )
    fig1.show()
except:

    k = horizon -2  # 0 = t+1, 1 = t+2
    # y_idx apunta al último target (t+h-1). Para k=0 con horizon=2, retrocede 1 posición.
    horizon = y.shape[1]
    offset = horizon - 1 - k  # = 1 si horizon=2 y k=0

    # Índices por split
    y_idx_train = y_idx[:n_train]
    y_idx_val   = y_idx[n_train:n_train+n_val]
    y_idx_test  = y_idx[n_train+n_val:n_train+n_val+n_test]

    # Construye un índice “k-correcto” desplazando dentro del índice completo de la serie
    all_idx = df_scaled.index  # índice cronológico base
    pos_test = all_idx.get_indexer(y_idx_test) - offset
    y_idx_test_k = all_idx[pos_test]

    # Series reales y predicciones para el paso k
    real_train = pd.Series(y_train[:, k], index=y_idx_train - pd.to_timedelta(offset, unit="D"), name="Real_train_k")
    real_val   = pd.Series(y_val[:,   k], index=y_idx_val   - pd.to_timedelta(offset, unit="D"), name="Real_val_k")
    real_test  = pd.Series(y_test[:,  k], index=y_idx_test_k, name="Real_test_k")

    # Predicciones:
    estatica_k    = pd.Series(y_pred_test[:, k], index=y_idx_test_k, name="Estatica_k")
    # Si tu walk-forward ya estaba tomando .ravel()[0] en cada paso, eso equivale a k=0:
    if y_pred_walk.ndim == 2 and y_pred_walk.shape[1] > 1:
        walk_k = pd.Series(y_pred_walk[:, k], index=y_idx_test_k, name="WalkForward_k")
    else:
        walk_k = pd.Series(np.ravel(y_pred_walk), index=y_idx_test_k, name="WalkForward_k")

    # Grafica
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=real_train.index, y=real_train.values, name="Real (train, t+1)"))
    fig1.add_trace(go.Scatter(x=real_val.index,   y=real_val.values,   name="Real (val, t+1)"))
    fig1.add_trace(go.Scatter(x=real_test.index,  y=real_test.values,  name="Real (test, t+1)"))
    fig1.add_trace(go.Scatter(x=estatica_k.index, y=estatica_k.values, name="Estática (t+1)"))
    fig1.add_trace(go.Scatter(x=walk_k.index,     y=walk_k.values,     name="Walk-Forward (t+1)"))

    fig1.update_layout(
        title="MSFT (scaled) — Real vs Estática vs Walk-Forward (paso t+1)",
        xaxis_title="Fecha", yaxis_title="price_scaled", hovermode="x unified",
        template="plotly_white"
    )
    fig1.show()


In [248]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=y_idx_train, y=y_train.ravel(), name="Real — Train", line=dict(width=1)))
fig1.add_trace(go.Scatter(x=y_idx_val,   y=y_val.ravel(),   name="Real — Val",   line=dict(width=1)))
fig1.add_trace(go.Scatter(x=y_idx_test,  y=y_test.ravel(),  name="Real — Test",  line=dict(width=2)))

fig1.add_trace(go.Scatter(x=y_idx_test, y=y_pred_test.ravel(),   name="Pred — Estática",     mode="lines"))
fig1.add_trace(go.Scatter(x=y_idx_test, y=y_pred_walk.ravel(),   name="Pred — Walk-Forward", mode="lines"))

fig1.update_layout(
    title="MSFT (scaled) — Real (train/val/test) vs Estática y Walk-Forward (test)",
    xaxis_title="Fecha", yaxis_title="price_scaled",
    hovermode="x unified", template="plotly_white"
)
fig1.show()

In [249]:
last_obs_date = df_scaled.index.max()
future_dates = pd.bdate_range(last_obs_date + BDay(1), periods=5)
future_dates

DatetimeIndex(['2025-10-20', '2025-10-21', '2025-10-22', '2025-10-23',
               '2025-10-24'],
              dtype='datetime64[ns]', freq='B')

## Predict FFNN

In [250]:
steps_ahead = 5
win = X[-1].copy()         
pred5_static = []

for _ in range(steps_ahead):
    yhat = model.predict(win.reshape(1, -1), verbose=1).ravel()[0]
    pred5_static.append(yhat)
    win = np.roll(win, -1) 
    win[-1] = yhat         

pred5_static = np.array(pred5_static)

df_future = pd.DataFrame(
    {"price_scaled_pred_static": pred5_static},
    index=future_dates
)

series_real = df_scaled["price_scaled"]
tail_n = 200 if len(series_real) > 200 else len(series_real)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


In [251]:
win_wf = X[-1].copy()  
pred5_wf = []
model_wf_last = models_used[-1]    

for _ in range(steps_ahead):
    yhat = model_wf_last.predict(win_wf.reshape(1, -1), verbose=0).ravel()[0]
    pred5_wf.append(yhat)
    win_wf = np.roll(win_wf, -1)
    win_wf[-1] = yhat

pred5_wf = np.array(pred5_wf)

df_future_wf = pd.DataFrame(
    {"price_scaled_pred_wf": pred5_wf},
    index=future_dates
)

In [254]:
display(df_future)
display(df_future_wf)

,price_scaled_pred_static
2025-10-20,0.607864
2025-10-21,0.589607
2025-10-22,0.592873
2025-10-23,0.583330
2025-10-24,0.585369


,price_scaled_pred_wf
2025-10-20,0.607864
2025-10-21,0.589607
2025-10-22,0.592873
2025-10-23,0.583330
2025-10-24,0.585369


In [252]:
series_real = df_scaled["price_scaled"]
tail_n = 200 if len(series_real) > 200 else len(series_real)

fig_future_wf = go.Figure()
fig_future_wf.add_trace(go.Scatter(
    x=series_real.index[-tail_n:], y=series_real.iloc[-tail_n:],
    name="Real (histórico)", mode="lines"
))
fig_future_wf.add_trace(go.Scatter(
    x=df_future.index, y=df_future["price_scaled_pred_static"],
    name="Forecast 5d — Estático", mode="lines+markers"
))
fig_future_wf.add_trace(go.Scatter(
    x=df_future_wf.index, y=df_future_wf.iloc[:, 0],  # price_scaled_pred_wf
    name="Forecast 5d — Walk-Forward", mode="lines+markers"
))

fig_future_wf.update_layout(
    title="MSFT (scaled) — Real (cola) + Forecast próximos 5 días (Estático vs Walk-Forward)",
    xaxis_title="Fecha", yaxis_title="price_scaled",
    hovermode="x unified", template="plotly_white"
)
fig_future_wf.show()

In [253]:
fig_future_wf_usd = go.Figure()
fig_future_wf_usd.add_trace(go.Scatter(
    x=series_real.index[-tail_n:], y=inv_scale(series_real.iloc[-tail_n:]),
    name="Real (USD, histórico)", mode="lines"
))
fig_future_wf_usd.add_trace(go.Scatter(
    x=df_future.index, y=inv_scale(df_future["price_scaled_pred_static"]),
    name="Forecast 5d — Estático (USD)", mode="lines+markers"
))
fig_future_wf_usd.add_trace(go.Scatter(
    x=df_future_wf.index, y=inv_scale(df_future_wf.iloc[:, 0]),
    name="Forecast 5d — Walk-Forward (USD)", mode="lines+markers"
))

fig_future_wf_usd.update_layout(
    title="MSFT — Real (cola) + Forecast 5 días en USD (Estático vs Walk-Forward)",
    xaxis_title="Fecha", yaxis_title="Precio (USD)",
    hovermode="x unified", template="plotly_white"
)
fig_future_wf_usd.show()


In [255]:
df_future_usd = df_future.copy()
df_future_usd.drop(columns=['price_scaled_pred_static'], inplace= True)
df_future_usd['price'] = scaler.inverse_transform(df_future[['price_scaled_pred_static']])
df_future_usd

,price
2025-10-20,511.845215
2025-10-21,510.737366
2025-10-22,510.935547
2025-10-23,510.356476
2025-10-24,510.480194
